In [55]:
import torch
from torch import nn
from torch.nn import *
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

df = pd.read_csv("mnist_train.csv")
df_test = pd.read_csv("mnist_test.csv")

y_df_train = df["label"]
X_df_train = df.drop('label', axis=1)
y_df_test = df_test["label"]
X_df_test = df_test.drop('label', axis=1)

y_train = torch.tensor(y_df_train.values)
X_train = torch.tensor(X_df_train.values).float()
y_test = torch.tensor(y_df_test.values)
X_test = torch.tensor(X_df_test.values).float()

In [68]:
'''class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
learning_rate = 1e-3
batch_size = 64
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)'''
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.conv1 = Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=2)
        
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=2)
        
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = Linear(in_features= 32 * 7 * 7, out_features=256)
        self.relu3 = ReLU()
        self.fc2 = Linear(in_features=256, out_features=10)
        # initialize our softmax classifier
        self.logSoftmax = LogSoftmax(dim=1)

        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        logits = self.logSoftmax(x)
        
        return logits

model = NeuralNetwork()
learning_rate = 1e-3
batch_size = 16
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [69]:
def train_loop(X_data, y_data, model, loss_fn, optimizer):
    size = len(X_data)
    X_data = X_data.reshape(60000, 1, 28, 28)
    X_split = torch.split(X_data, batch_size, dim=0)
    y_split = torch.split(y_data, batch_size, dim=0)
    for X, y in zip(X_split, y_split):
       # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_size % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test_loop(X, y, model, loss_fn):
    size = len(X)
    num_batches = size / batch_size
    test_loss, correct = 0, 0
    with torch.no_grad():
        X = X.reshape(10000, 1, 28, 28)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    loss_fn = nn.CrossEntropyLoss()    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [70]:
epochs = 100
for t in range(epochs):
    print("Epoch {t+1}\n-------------------------------")
    train_loop(X_train, y_train, model, loss_fn, optimizer)
    test_loop(X_test, y_test, model, loss_fn)
print("Done!")

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 97.7%, Avg loss: 0.000104 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.3%, Avg loss: 0.000075 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000063 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000059 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.000055 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.000056 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.000060 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.000064 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.000067 

Epoch {t+1}
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000070 

Epoch {t+1}
----------------------------

KeyboardInterrupt: 

In [67]:
from PIL import Image

test_num = 0
X_test = X_test.reshape(10000, 1, 28, 28)
pred = model(X_test)

while(test_num < 60000):
    test_num += 1
    if(pred[test_num].argmax(0) != y_df_test[test_num]):
        if(input("Would you like to see the next failed Image? (y/n): ") == "y"):
            X_test_img = X_df_test.values[test_num ].reshape(28, 28)
            X_test_img = np.array(X_test_img).astype(np.uint8) # Create a PIL image

            img = Image.fromarray( X_test_img )
            img.show() 
    
            print("Test Number " + str(test_num) + " failed: ")
            print(pred[test_num])
            print(y_test[test_num])
        else:
            break



Would you like to see the next failed Image? (y/n): y
Test Number 18 failed: 
tensor([-17.8222, -29.0882, -17.8921,  -1.7103, -51.7656,  -0.1995, -33.1545,
        -31.2344, -13.8431, -41.9014], grad_fn=<SelectBackward0>)
tensor(3)
Would you like to see the next failed Image? (y/n): y
Test Number 62 failed: 
tensor([-28.3925, -36.9957, -42.5317, -37.8827, -27.9764,  -0.0905, -26.5295,
        -26.5592, -13.2476,  -2.4469], grad_fn=<SelectBackward0>)
tensor(9)
Would you like to see the next failed Image? (y/n): y
Test Number 92 failed: 
tensor([-31.8822, -26.1097, -33.2499, -21.3225,  -7.7039, -22.1045, -27.9171,
        -20.9024,  -0.0948,  -2.4081], grad_fn=<SelectBackward0>)
tensor(9)
Would you like to see the next failed Image? (y/n): y
Test Number 321 failed: 
tensor([-3.3188e+01, -2.8950e+01, -3.0762e+00, -1.7414e+01, -4.9753e+01,
        -5.3329e+01, -6.5552e+01, -4.7233e-02, -1.8610e+01, -3.2844e+01],
       grad_fn=<SelectBackward0>)
tensor(2)
Would you like to see the next fai